In [593]:
import pandas as pd
from datetime import datetime, date
import random

In [388]:
class Account:
    Database = []
    ID = 1
    
    def __init__(self, username):
        self.username = username
        
    def save(self, date=datetime.now().date()):
        if self.username in [akun['username'] for akun in Account.Database]:
            message = "Account already exists."
            print(message)
            return message
        data = {
            "id": Account.ID,
            "username": self.username,
            "date created": date,
            "followings": [],
            "followers": []
        }
        Account.Database.append(data)
        Account.ID += 1
        
    def get_account(self):
        """
            if found, returns index of the account in the List Database
            and the Account dictionary
        """
        for i in range(len(Account.Database)):
            if Account.Database[i]['username'] == self.username:
                return i, Account.Database[i]
        print(f"Account {self.username} not in Database")
        return "No index.", "Account not found."
        
    def add_following(self, username):
        """
            Adding follwing.
            arg: Class username within database
        """
        if username.username == self.username:
            message = "Can't follow ownself."
            print(message)
            return message
        index, akun = self.get_account()
        if username.username in [account.username for account in akun['followings']]:
            message = f"{username.username} is already followed by {self.username}"
            print(message)
            return message
        akun['followings'].append(username)
        Account.Database[index] = akun
        username._add_follower(self)
        
    def _add_follower(self, username):
        """
            Adding follower.
            arg: Class username within database
        """
        if username.username == self.username:
            message = "Can't follow ownself."
            print(message)
            return message
        index, akun = self.get_account()
        if username.username in [account.username for account in akun['followers']]:
            message = f"{username.username} is already following {self.username}"
            print(message)
            return message
        akun['followers'].append(username)
        Account.Database[index] = akun
        
    def following(self):
        index, akun = self.get_account()
        return akun['followings']
    
    def follower(self):
        index, akun = self.get_account()
        return akun['followers']
        
    def __repr__(self):
        return f"{self.username}"
    
    def account_total():
        return len(Account.Database)
    
    def database_reset():
        Account.ID = 0
        Account.Database = []
        return "OK"


In [450]:
class Post:
    Database = []
    ID = 1
    
    def __init__(self, author):
        assert isinstance(author, Account)
        self.author = author
        
    def save(self, date=datetime.now()):
        index, akun = self.author.get_account()
        # if date for the post is before the account created which doesn't any make sense
        if date.date() < akun["date created"]:
            date = datetime.now()
        data = {
            "id": Post.ID, # integer
            "author": self.author, # class
            "date": date, # datetime
            "comments": [], # list of sets of komentar
        }
        Post.Database.append(data)
        Post.ID += 1
    
    def get_posts(self):
        """
            if found, returns all the posts made by the 
            predefined author
        """
        posts = []
        for i in range(len(Post.Database)):
            if Post.Database[i]['author'].username == self.author.username:
                posts.append(Post.Database[i])
        return posts
        
    def add_comment(post_id, account, date=datetime.now()):
        """
            Add comment
            arg: Class account
            We won't pay attention to the comment's detail like the content,
            hence only account data and date data are saved.
        """
        assert isinstance(account, Account)
        index = post_id - 1
        post = Post.Database[index]
        # if date for the comment is before the post was made
        if date < Post.Database[index]['date']:
            date = datetime.now()
        komentar = (account, date)
        post['comments'].append(komentar)
        Post.Database[index] = post
        
    def latest_post(self):
        posts = self.get_posts()
        if any(posts):
            return posts[-1]
        return "This user hasn't made any posts yet."
    
    def get_comments(self):
        comments = []
        return comments
    
    def database_reset():
        Post.ID = 0
        Post.Database = []
        return "OK"

In [451]:
# import original data
df = pd.read_excel("LIST FOLLOWING IG.xlsx", usecols=[1, 2, 3, 4, 5, 6])
df.head(5)

,rinogrego,knchshen,devirynt_,anryh_,mahiramarsha,saaalaaaz
0,fabianandhika,mnurichisan,fairuziazahira,ajilanaqila,devirynt_,vaniaadisaa
1,fairuziazahira,amandaind_11,kint.ann,fairuziazahira,zalfaharis,amiirdhia
2,rayhanfadilla,hanifdnswr,aprll.liaa,dio_ahnaf,clarissans_,farah_august
3,darindhiani,shanifah1110,karinmrsh,ayumnaf,sulthan_ali_pasha,yusufbeltsazarr
4,daffaadra,danielrbrto,amanda.si,aamnd_82,pusvitanf,divinadira


In [452]:
# Define unique users
unique_users = []

for col in df.columns:
    for row in df[col]:
        if row not in unique_users and row == row:
            unique_users.append(row)


## Create Accounts Dictionary 
<br></br>
#### key: username string
#### value: Account object of corresponding username

In [453]:
# Initiate unique users as dictionary of objects
user_objects = {}

# reset database
Account.database_reset()

for account in unique_users:
    user_objects[account] = Account(account)
    # randomize account creation date
    year = random.randrange(2017, 2022)
    month = random.randrange(1, 13)
    if month in [1, 3, 5, 7, 8, 10, 12]:
        day = random.randrange(1, 32)
    elif month in [4, 6, 9, 11]:
        day = random.randrange(1, 31)
    elif month is 2:
        day = random.randrange(1, 29)
    date = datetime(year, month, day).date()
    user_objects[account].save(date=date)

# Account.Database

## Generate Following Data
<br></br>
#### key: username string
#### value: list of following of corresponding user

In [454]:
# generate following data
users_following = {}

# original data
for col in df.columns:
    users_following[col] = [row for row in df[col]]
    
# synthetic data
for account in unique_users:
    if account in users_following:
        continue
    # randomize number of following
    number_of_following = random.randrange(0, len(unique_users)-1)
    
    # define list of users that can be followed
    following_list = []
    users_to_follow = unique_users[:]
    users_to_follow.remove(account)
    
    # randomize the users followed
    for following in range(number_of_following):
        random_user = random.choice(users_to_follow)
        following_list.append(random_user)
        users_to_follow.remove(random_user)
        
    users_following[account] = following_list


In [455]:
# create following database
for username, user_object in user_objects.items():
    for user_to_follow in users_following[username]:
        user_to_follow_object = user_objects[user_to_follow] # get the Account object of the user
        user_object.add_following(user_to_follow_object)

## Generate Post Data

In [473]:
# reset first, or can be commented
Post.database_reset()

for username, user_object in user_objects.items():
    # randomize number of posts the user made
    number_of_posts = random.randrange(0, 100)
    # create random posts
    for num in range(number_of_posts):
        # randomize post creation date
        year = random.randrange(2017, 2022)
        month = random.randrange(1, 13)
        if month in [1, 3, 5, 7, 8, 10, 12]:
            day = random.randrange(1, 32)
        elif month in [4, 6, 9, 11]:
            day = random.randrange(1, 31)
        elif month is 2:
            day = random.randrange(1, 29)
        date = datetime(year, month, day)
        Post(user_object).save(date=date)
        

In [474]:
len(Post.Database)

7873

## Generate Comment Data

In [475]:
# add comment towards a post id by Account object

for username, user_object in user_objects.items():
    number_of_comments_user_made = random.randrange(0, 500)
    for num in range(number_of_comments_user_made):
        post_for_the_comment_index = random.randrange(1, len(Post.Database)+1) # pick random post to comment
        # randomize post creation datetime
        year = random.randrange(2017, 2022)
        month = random.randrange(1, 13)
        if month in [1, 3, 5, 7, 8, 10, 12]:
            day = random.randrange(1, 32)
        elif month in [4, 6, 9, 11]:
            day = random.randrange(1, 31)
        elif month is 2:
            day = random.randrange(1, 29)
        date = datetime(year, month, day)
        Post.add_comment(post_for_the_comment_index, user_object, date=date)

In [522]:
Post.Database[2859]

{'id': 2859,
 'author': rinogrego,
 'date': datetime.datetime(2021, 12, 19, 21, 34, 0, 17570),
 'comments': [(officialbayuj,
   datetime.datetime(2021, 12, 19, 21, 34, 3, 921322)),
  (___widi__, datetime.datetime(2021, 12, 19, 21, 34, 3, 972871)),
  (agungyudhis, datetime.datetime(2021, 12, 19, 21, 34, 4, 6413)),
  (sesarsatyarini, datetime.datetime(2021, 12, 19, 21, 34, 4, 53473)),
  (maxwelth, datetime.datetime(2021, 12, 19, 21, 34, 4, 74994)),
  (alfinawi, datetime.datetime(2021, 12, 19, 21, 34, 4, 80502)),
  (stephanieodelia, datetime.datetime(2021, 12, 19, 21, 34, 4, 239035)),
  (eslimsuyangsu, datetime.datetime(2021, 12, 19, 21, 34, 4, 329088))]}

## Algorithm

In [550]:
def get_weight(user):
    """
        Calculate the user's weight based on
        its latest post date
        weight = 1                      , if days_since_last_post is <= 3
        weight = 1 / (days - 3)**(1/3)  , if days_since_last_post is > 3
        input: user (Account object)
        output: weight
    """
    now = datetime.now().date()
    try:
        latest_post_date = Post(user).latest_post()['date'].date()
    except:
        index, akun = user.get_account()
        latest_post_date = akun['date created']
    days = (now - latest_post_date).days
    
    weight = 1
    if days > 3:
        weight *= (1 / (days - 3))**(1/3)
    return weight


def target_activity_value(target, weight):
    """
        Giving additional weights for the target based on
        the number of posts made by the target
        input: target (Account object), weight (of the target)
        output: target, new weight
    """
    posts_total = len(Post(target).get_posts())
    weight += posts_total * 0.05
    return target, weight


def interaction_value(user, target, weight):
    """
        Check interactions between user Account and target Account
        then give numerical value to it.
        if target commented on a user's post, then the weight is increased by 0.02 / comment
        If user commented on a target's post, then the weight is increased by 0.05 / comment
        input: user (Account object), target (Account object), weight (of the target)
        output: target, new weight
    """
    
    # check all the comments made by target Account to all of the user Account's posts.
    for post in Post(user).get_posts():
        if post['comments'] == []: # if no comment exists in the post, then continue
            continue
        for comment in post['comments']:
            if comment[0] == target:
                weight += 0.02
                
    # check all the comments made by user Account to all of the target Account's posts.
    for post in Post(target).get_posts():
        if post['comments'] == []: # if no comment exists in the post, then continue
            continue
        for comment in post['comments']:
            if comment[0] == user:
                weight += 0.05
        
    return target, weight
    

def get_recommendation(user, maximum=20, recommend_followers=False):
    """
        Get recommendation of accounts for user to follow
        Recommendation is based upon weights calculated based on some factors:
            - accounts that follows the user (user's follower)
            - accounts that followed by the user's followings
            - activity of the target account (last post date, or if none exists then account's date creation)
            - number of interactions b/w the user and the account targets, 
                - weight of user commenting on a target's post > weight of target commenting on a user's post
                - each interaction is counted as an addition to already defined list of recommended accounts
                - if user commented on a user A's post, but user A is not in the list of potential accounts to follow, then
                    the number of interactions won't be counted. may be fixed next.
        input: user (Account object), maximum (integer), recommend_followers (boolean)
        output: following_potential (dictionary)
    """
    following_potential = {}
    
    # initialize weight for all the followers of the user
    # can be omitted from the algorithm
    if recommend_followers == True:
        for akun in user.follower():
            W = get_weight(akun)
            following_potential[akun] = W
    
    # finding common following
    # depth-first search
    for akun in user.following():
        for akun2 in akun.following():
            # if target is already followed or target is the user itself
            if akun2 in user.following() or akun2 == user:
                continue
            W = get_weight(akun2)
            # if common following has been found
            if akun2 in following_potential:
                following_potential[akun2] *= 1.2
                continue
            following_potential[akun2] = W
    
    # check user interaction with targets in the following potential
    for target, weight in following_potential.items():
        target, weight = target_activity_value(target, weight) # calculate the target's weight based on its activity
        target, weight = interaction_value(user, target, weight) # calculate the target's weight based on the interactions happened with the user
        following_potential[target] = weight
        
    # sorting algorithm
    following_potential = {target: weight for target, weight in sorted(following_potential.items(), key=lambda item: item[1], reverse=True)}
    
    # only get maximum number of recommendation
    best_following_potential = {}
    i = 0
    for key, value in following_potential.items():
        i += 1
        best_following_potential[key] = value
        if i == maximum:
            break
        
    return best_following_potential

## Get Recommendation

In [596]:
user = Account('rinogrego')
get_recommendation(user)

{yudhafernandoo: 344.02189187166823,
 alfinawi: 237.99631379976964,
 tiaraayumi_: 198.1335948331414,
 hanadzania: 169.44466236095113,
 rayhan.adi5: 166.16466236095116,
 vincentm28: 165.44466236095113,
 farah_august: 141.4205519674593,
 michael_bram10: 139.24055196745928,
 deandrasp: 138.47055196745927,
 sharifamira_: 138.02055196745928,
 vaniaadisaa: 117.72545997288273,
 ervitaindahpratiwi: 117.32545997288273,
 sagitratrimeizanda: 117.22545997288273,
 benedictawinni: 116.02545997288273,
 darinr.a: 115.07545997288273,
 ucharusyana: 114.92545997288273,
 stephanieodelia: 99.78621664406893,
 nisa.nh_: 99.49621664406894,
 jilanaqila: 98.84621664406895,
 ridhoelfas: 98.81621664406894}

# Dibuat oleh: Gregorino Al Josan